<a href="https://colab.research.google.com/github/jeongeun/ROOTExamples/blob/master/Testcode_TripleElectron_TTbar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install uproot
!pip install glob
!pip install awkward
!pip install  numpy
!pip install subprocess
!pip install vector
!pip install  matplotlib
!pip install  mplhep

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
ERROR: Could not find a version that satisfies the requirement subprocess (from versions: none)
ERROR: No matching distribution found for subprocess


In [4]:
import uproot as up
import awkward as ak
import numpy as np
from itertools import combinations
from tqdm import tqdm
import time

start_time = time.time()

In [5]:
infile = "/content/drive/MyDrive/Colab/ntuple_mc_130X_2023_BPix_125902_0.root:ntuple/tree"  # TTbar 13.6TeV 23 post sample
f = up.open(infile)
tree = f.arrays(filter_name=["Ele_pt", "Ele_etaSC", "Ele_phiSC", "Ele_Et"])
Ele = ak.zip(
{
      "pt"   : tree["Ele_pt"],
      "eta"  : tree["Ele_etaSC"],
#      "phi"  : tree["Ele_phiSC"],
#      "Et"   : tree["Ele_Et"],
 })

def cut(candidates): #already cut applied at masking # 5
       return candidates[candidates.pt >= 30.]

pt15_Ele = cut(Ele)
print("@ electron before any cut = : ",len(Ele.pt), " , Et= ", Ele.pt, ", ak.num(Ele)= ", ak.num(Ele))
print("@ electron pt > 15GeV cut = :", len(pt15_Ele.pt), " , Et= ", pt15_Ele.pt, ", ak.num(Ele)= ", ak.num(pt15_Ele))

@ electron before any cut = :  48000  , Et=  [[30.2, 15.6], [], [], [37.8], ..., [...], [40.7], [60.9, 13.9], [25.7, 24.9]] , ak.num(Ele)=  [2, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 2, ..., 0, 1, 1, 2, 0, 1, 0, 0, 2, 1, 2, 2]
@ electron pt > 15GeV cut = : 48000  , Et=  [[30.2], [], [], [37.8], [], [], [], ..., [], [], [], [40.7], [60.9], []] , ak.num(Ele)=  [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, ..., 0, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0]


In [6]:
# Make triple electron set
enum_mask = (ak.num(pt15_Ele) >= 3)
filtered_Ele = pt15_Ele[enum_mask]

print("@ electron num >=3 cut = :", len(filtered_Ele.pt), " , Et= ", filtered_Ele.pt, ", ak.num(Ele)= ", ak.num(filtered_Ele))

@ electron num >=3 cut = : 289  , Et=  [[100, 45.9, 37.1], [73.1, 31.6, 31], ..., [44.9, ..., 39.8], [125, 61.2, 51]] , ak.num(Ele)=  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..., 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [7]:
# Efficiently generate combinations and count them per event
# and select the best one per event

combination_counts = []   # 한 이벤트 당 (e,e,e) 조합 갯수 체크해보기
all_comb_eee       = []   # 한 이벤트에서 가능한 모든 조합 저장
best_comb_eee      = []   # 한이벤드 당 딱 하나 최고의 조합 저장!!

for event in tqdm(filtered_Ele):
    comb_eee = list(combinations(event, 3))
    combination_counts.append(len(comb_eee))
    all_comb_eee.extend(comb_eee) # 모든 조합을 all_comb_eee에 저장.

    # select the best combination per event
    if len(comb_eee) == 1:
        best_comb_eee.append(comb_eee[0])
    else: # 하나 최고의 조합을 찾는 파트 (max pt sum으로 선택)
        max_pt_sum = 0
        best_comb = None
        for comb in comb_eee:
            leading_ele, second_ele, third_ele = comb
            pt_sums = [
                leading_ele.pt + second_ele.pt,
                leading_ele.pt + third_ele.pt,
                second_ele.pt + third_ele.pt
            ]
            max_pt_sum_comb = max(pt_sums)
            if max_pt_sum_comb > max_pt_sum:
                max_pt_sum = max_pt_sum_comb
                best_comb = comb
        best_comb_eee.append(best_comb)

# Converting to an awkward array for efficient storage and usage
all_comb_eee = ak.from_iter(all_comb_eee) # 확인용

best_comb_eee = ak.from_iter(best_comb_eee) #최종 선택한 조합

end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")


100%|██████████| 289/289 [00:00<00:00, 9429.14it/s]

Time taken: 0.33791089057922363 seconds


In [12]:
# Print the combination counts
print("Combination counts per event:", combination_counts)
print("Total number of combinations:", sum(combination_counts))

print("==> Total number of selected combinations:", len(best_comb_eee))
# 이벤트 수와 정확히 동일함을 확인.

Combination counts per event: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Total number of combinations: 337
==> Total number of selected combinations: 289


In [14]:
# Print the type of the array to see the structure
print(ak.type(all_comb_eee))
print("-------")
# Show the structure with some example data
print(all_comb_eee.show())

337 * ({pt: float64, eta: float64}, {pt: float64, eta: float64}, {pt: float64, eta: float64})
-------
[({pt: 100, eta: 1.97}, {pt: 45.9, eta: ..., ...}, {...}),
 ({pt: 73.1, eta: 0.602}, {pt: 31.6, ...}, {pt: 31, ...}),
 ({pt: 90.2, eta: 0.611}, {pt: 45.9, ...}, {pt: 35.6, ...}),
 ({pt: 145, eta: -0.642}, {pt: 39.6, ...}, {pt: 30.4, ...}),
 ({pt: 84.7, eta: 0.033}, {pt: 84.2, ...}, {pt: 57.6, ...}),
 ({pt: 69.5, eta: 0.408}, {pt: 32.5, ...}, {pt: 30.7, ...}),
 ({pt: 50, eta: 1.14}, {pt: 31.9, eta: ..., ...}, {...}),
 ({pt: 82.4, eta: 0.325}, {pt: 80.9, ...}, {pt: 60.1, ...}),
 ({pt: 105, eta: -1.73}, {pt: 43, eta: 1.02}, {pt: ..., ...}),
 ({pt: 80.7, eta: -1.69}, {pt: 71.9, ...}, {pt: 48.9, ...}),
 ...,
 ({pt: 115, eta: -0.399}, {pt: 75.3, ...}, {pt: 54.2, ...}),
 ({pt: 55.4, eta: 0.0882}, {pt: 44.7, ...}, {pt: 33.6, ...}),
 ({pt: 65, eta: 0.871}, {pt: 62.4, eta: ..., ...}, {...}),
 ({pt: 130, eta: 1.26}, {pt: 123, eta: ..., ...}, {...}),
 ({pt: 161, eta: -0.229}, {pt: 159, eta: ..., .

In [15]:
# Print the type of the array to see the structure
print(ak.type(best_comb_eee))
print("-------")
# Show the structure with some example data
print(best_comb_eee.show())


289 * ({pt: float64, eta: float64}, {pt: float64, eta: float64}, {pt: float64, eta: float64})
-------
[({pt: 100, eta: 1.97}, {pt: 45.9, eta: ..., ...}, {...}),
 ({pt: 73.1, eta: 0.602}, {pt: 31.6, ...}, {pt: 31, ...}),
 ({pt: 90.2, eta: 0.611}, {pt: 45.9, ...}, {pt: 35.6, ...}),
 ({pt: 145, eta: -0.642}, {pt: 39.6, ...}, {pt: 30.4, ...}),
 ({pt: 84.7, eta: 0.033}, {pt: 84.2, ...}, {pt: 57.6, ...}),
 ({pt: 69.5, eta: 0.408}, {pt: 32.5, ...}, {pt: 30.7, ...}),
 ({pt: 50, eta: 1.14}, {pt: 31.9, eta: ..., ...}, {...}),
 ({pt: 82.4, eta: 0.325}, {pt: 80.9, ...}, {pt: 60.1, ...}),
 ({pt: 105, eta: -1.73}, {pt: 43, eta: 1.02}, {pt: ..., ...}),
 ({pt: 80.7, eta: -1.69}, {pt: 71.9, ...}, {pt: 48.9, ...}),
 ...,
 ({pt: 115, eta: -0.399}, {pt: 75.3, ...}, {pt: 54.2, ...}),
 ({pt: 55.4, eta: 0.0882}, {pt: 44.7, ...}, {pt: 33.6, ...}),
 ({pt: 65, eta: 0.871}, {pt: 62.4, eta: ..., ...}, {...}),
 ({pt: 130, eta: 1.26}, {pt: 123, eta: ..., ...}, {...}),
 ({pt: 161, eta: -0.229}, {pt: 159, eta: ..., .

In [ ]:
# Print the first few combinations to inspect
print(ak.to_list(all_comb_eee[:2]))

In [ ]:
# Optionally save the combinations to a file
ak.to_parquet(all_comb_eee, "triple_electron_combinations.parquet")